In [ ]:
# Metric Evaluation for the query expansion approach.
# 1. Loading the data from the touche topics file.
# ---
# ---
# IMPORTANT, relevant for script: We perform multiple retrievals for all the expanded queries.
# Note: This might be an important modification required from the current script perspective.

# Second, we use retrieval of our main query as driving matcher, and check whether for each query
# the document is retrieved by any other of the pooled queries (at least 1, threshold).
# If yes, we keep those documents. (We do it for first 1K documents, but we will essentially end up with ,1k documents)

# After this, we select first-k relevant documents from each of the pool queries 
# The selected number of retrieved documents is optimized based on practical experience with retrieval count tuning.

# Finally, those first-k relevant documents that are not present in our list from the second step are appended until 1.5k
# document count is reached. This logic is implemented in the form of a dictionary iterator in the upcoming cells.
# ---
# ---
# 2. Loading the corresponding queries into List, making predictions of k=275 (min) on the built index.
# 3. Merging Logic: For additional queries, remove the matching documents & keep only first non-existing docs.
# 4. Evaluation Metric: Average coverage/recall calculation for the voting based query expansion.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import statements for retrieval evaluation.
import pandas as pd

In [26]:
# not relevant for the script, as you'll be getting pool of expanded from the above function.
touche_topics = pd.read_csv('/content/drive/MyDrive/Touche/touche_topics_query_expansion.csv')
touche_topics.head()

,number,title,query_base,query_noun,query_synonym1,query_synonym2,query_synonym3,query_antonym1,query_antonym2
0,1,What is the difference between sex and love?,difference sex and love,difference sex love,different difference sex love,unlike difference sex love,dissimilar difference sex love,bad difference sex love,worse difference sex love
1,2,"Which is better, a laptop or a desktop?",better laptop or desktop,laptop desktop,good laptop desktop,well laptop desktop,improve laptop desktop,worse laptop desktop,different laptop desktop
2,3,"Which is better, Canon or Nikon?",better Canon or Nikon,Canon Nikon,good Canon Nikon,well Canon Nikon,improve Canon Nikon,worse Canon Nikon,different Canon Nikon
3,4,What are the best dish detergents?,best dish detergents,detergents,better detergents,good detergents,well detergents,worst detergents,worse detergents
4,5,What are the best cities to live in?,best cities live,cities live,better cities,good cities,well cities,worst cities,worse cities


In [27]:
psuedo=pd.read_csv('/content/drive/MyDrive/Touche/touche_complete_topics_psuedo.csv')
ground_truth=pd.read_csv("/content/drive/MyDrive/Touche/touche_complete_topics.csv")
psuedo.head()
new=ground_truth.join(psuedo.set_index("Original"),on="Title")

In [28]:
new

,Number,Title,Unnamed: 0,New,Frequency
0,1,\nWhat is the difference between sex and love?\n,0,\nWhat is the difference between sex and love?...,18
1,2,"\nWhich is better, a laptop or a desktop?\n",1,"\nWhich is better, a laptop or a desktop?\n home",9
2,3,"\nWhich is better, Canon or Nikon?\n",2,"\nWhich is better, Canon or Nikon?\n vs",29
3,4,\nWhat are the best dish detergents?\n,3,\nWhat are the best dish detergents?\n brands,35
4,5,\nWhat are the best cities to live in?\n,4,\nWhat are the best cities to live in?\n places,13
...,...,...,...,...,...
95,96,"Which is healthier to wear, boxers or briefs?",95,"Which is healthier to wear, boxers or briefs? men",16
96,97,What is the difference between a blender vs a ...,96,What is the difference between a blender vs a ...,19
97,98,"Which is better, rock or rap?",97,"Which is better, rock or rap? music",29
98,99,Do you think imagination is better than knowle...,98,Do you think imagination is better than knowle...,13


In [29]:
touche_topics=touche_topics.join(new.set_index("Number"),on="number")
touche_topics

,number,title,query_base,query_noun,query_synonym1,query_synonym2,query_synonym3,query_antonym1,query_antonym2,Title,Unnamed: 0,New,Frequency
0,1,What is the difference between sex and love?,difference sex and love,difference sex love,different difference sex love,unlike difference sex love,dissimilar difference sex love,bad difference sex love,worse difference sex love,\nWhat is the difference between sex and love?\n,0,\nWhat is the difference between sex and love?...,18
1,2,"Which is better, a laptop or a desktop?",better laptop or desktop,laptop desktop,good laptop desktop,well laptop desktop,improve laptop desktop,worse laptop desktop,different laptop desktop,"\nWhich is better, a laptop or a desktop?\n",1,"\nWhich is better, a laptop or a desktop?\n home",9
2,3,"Which is better, Canon or Nikon?",better Canon or Nikon,Canon Nikon,good Canon Nikon,well Canon Nikon,improve Canon Nikon,worse Canon Nikon,different Canon Nikon,"\nWhich is better, Canon or Nikon?\n",2,"\nWhich is better, Canon or Nikon?\n vs",29
3,4,What are the best dish detergents?,best dish detergents,detergents,better detergents,good detergents,well detergents,worst detergents,worse detergents,\nWhat are the best dish detergents?\n,3,\nWhat are the best dish detergents?\n brands,35
4,5,What are the best cities to live in?,best cities live,cities live,better cities,good cities,well cities,worst cities,worse cities,\nWhat are the best cities to live in?\n,4,\nWhat are the best cities to live in?\n places,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,"Which is healthier to wear, boxers or briefs?",healthier wear boxers or briefs,wear boxers briefs,healthy boxers briefs,fitter boxers briefs,salubrious boxers briefs,unhealthy boxers briefs,worse boxers briefs,"Which is healthier to wear, boxers or briefs?",95,"Which is healthier to wear, boxers or briefs? men",16
96,97,What is the difference between a blender vs a ...,difference blender food processor,difference blender food processor,good difference blender food processor,well difference blender food processor,improve difference blender food processor,worse difference blender food processor,different difference blender food processor,What is the difference between a blender vs a ...,96,What is the difference between a blender vs a ...,19
97,98,"Which is better, rock or rap?",better rock or rap,rock rap,good rock rap,well rock rap,improve rock rap,worse rock rap,different rock rap,"Which is better, rock or rap?",97,"Which is better, rock or rap? music",29
98,99,Do you think imagination is better than knowle...,think imagination better knowledge,think imagination knowledge,believe imagination knowledge,guess imagination knowledge,imagine imagination knowledge,forget imagination knowledge,worse imagination knowledge,Do you think imagination is better than knowle...,98,Do you think imagination is better than knowle...,13


In [30]:
# not relevant for the script, as you'll be getting pool of expanded from the above function.
queries = touche_topics['title']
base_queries = touche_topics['New']
noun_queries = touche_topics['query_noun']
syn1_queries = touche_topics['query_synonym1']
syn2_queries = touche_topics['query_synonym2']
syn3_queries = touche_topics['query_synonym3']
ant1_queries = touche_topics['query_antonym1']
ant2_queries = touche_topics['query_antonym2']

In [ ]:
# important for making the 'pyserini' library work.
# installing linux related stuff for pyserini
!sudo apt-get install libomp-dev
# installing important packages for building the new index on merged documents.
!pip install pyserini
!pip install faiss

In [32]:
# relevant for the script, loading the already stored and built index.
from pyserini.search.lucene import LuceneSearcher
searcher_opt = LuceneSearcher('/content/drive/MyDrive/Touche/sample_collection_jsonl')
searcher_opt.set_bm25(1.2, 0.68)

In [33]:
# relevant for the script, getting the driving query retrieval in the solution dictionary.
solution_dict = {}
for id_, q_ in zip(touche_topics['number'], queries):
    hits = searcher_opt.search(q_.strip(), k=1500)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict[id_] = d_list

In [34]:
# relevant for the script, getting the noun query retrievals.
solution_dict_noun = {}
for id_, q_ in zip(touche_topics['number'], noun_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_noun[id_] = d_list

In [35]:
# relevant for the script, getting the base query retrievals.
solution_dict_base = {}
for id_, q_ in zip(touche_topics['number'], base_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_base[id_] = d_list

In [36]:
# relevant for the script, getting the syn1 query retrievals.
solution_dict_syn1 = {}
for id_, q_ in zip(touche_topics['number'], syn1_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_syn1[id_] = d_list

In [37]:
# relevant for the script, getting the syn2 query retrievals.
solution_dict_syn2 = {}
for id_, q_ in zip(touche_topics['number'], syn2_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_syn2[id_] = d_list

In [38]:
# relevant for the script, getting the syn3 query retrievals.
solution_dict_syn3 = {}
for id_, q_ in zip(touche_topics['number'], syn3_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_syn3[id_] = d_list

In [39]:
# relevant for the script, getting the ant1 query retrievals.
solution_dict_ant1 = {}
for id_, q_ in zip(touche_topics['number'], ant1_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_ant1[id_] = d_list

In [40]:
# relevant for the script, getting the ant2 query retrievals.
solution_dict_ant2 = {}
for id_, q_ in zip(touche_topics['number'], ant2_queries):
    hits = searcher_opt.search(q_.strip(), k=3000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict_ant2[id_] = d_list

In [41]:
# IMPORTANT: Main document merging logic, you might require a function or I'll recommend straight up adding it in the voting/expansion script.
# Solution dictionary final below, will store list of all document (but not in ordered manner) to be fed into monoT5.
# The order doesn't matter as monoT5 ingnores all the associated scores.
solution_dict_fin = {}
# iterating over all the pooled queries that we have created, plus our driving query.
for (k,v), (k1,v1), (k2,v2), (k3,v3), (k4,v4), (k5,v5), (k6,v6), (k7,v7) \
    in zip(solution_dict.items(), solution_dict_noun.items(), solution_dict_base.items(), solution_dict_syn1.items(), \
           solution_dict_syn2.items(), solution_dict_syn3.items(), solution_dict_ant1.items(), solution_dict_ant2.items()):    
    # check for all k's being equal is quite unnecessary though.
    if k == k1 and k1 == k2 and k2 == k3 and k3 == k4 and k4 == k5 and k5 == k6 and k6 == k7:
        # for storing the list of 1500 relevant documents as per our merging logic.
        l_temp = []

        # finding commonality amongst documents from different queries.
        # below 'v' represents documents from driving main query.
        # we essentially check, whether the given document v_ in v is present
        # in any of the other retrievals by other pool of expanded queries.
        # If, v_ is present in any of the documents we add it into the l_temp.
        v_x = v[:1000] # limiting only to 1500 entries only.
        for v_ in v_x:
            c_b = 0
            if v_ in v1:
                c_b = c_b + 1
            if v_ in v2:
                c_b = c_b + 1
            if v_ in v3:
                c_b = c_b + 1
            if v_ in v4:
                c_b = c_b + 1
            if v_ in v5:
                c_b = c_b + 1
            if c_b >= 1:
                l_temp.append(v_)

        # for the query we will append documents
        # till it reaches the 1125 document count for l_temp.
        diff_ = 1125 - len(l_temp)
        
        # appending document logic.
        c_ = 0
        for v1_ in v1: # appending base query documents.
            if v1_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v1_)
            elif c_ > diff_:
                break
            c_ = c_ + 1

        c_ = 0
        for v2_ in v2: # appending noun query documents.
            if v2_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v2_)
            elif c_ > 150:
                break
            c_ = c_ + 1

        c_ = 0
        for v3_ in v3: # appending syn1 documents.
            if v3_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v3_)
            elif c_ > 75:
                break
            c_ = c_ + 1

        c_ = 0
        for v4_ in v4: # appending syn2 documents.
            if v4_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v4_)
            elif c_ > 46:
                break
            c_ = c_ + 1

        c_ = 0
        for v5_ in v5: # appending syn3 documents.
            if v5_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v5_)
            elif c_ > 45:
                break
            c_ = c_ + 1

        c_ = 0
        for v6_ in v6: # appending ant documents.
            if v6_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v6_)
            elif c_ > 30:
                break
            c_ = c_ + 1

        c_ = 0
        for v7_ in v7: # appending ant documents.
            if v7_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v7_)
            elif c_ > 30:
                break
            c_ = c_ + 1
        
        c_ = 0
        for v8_ in v_x: # appending org. documents.
            if v8_ not in l_temp: # only relevant non-existing docs added.
                l_temp.append(v8_)

        solution_dict_fin[k] = l_temp[:1500]

In [42]:
# Testing out the number of documents in the retrieval.
i = 0
for x, y in solution_dict_fin.items():
    print(x, len(y))
    if i > 10:
        break
    i = i+1

1 1009
2 1000
3 1000
4 1040
5 1230
6 1288
7 1015
8 1037
9 1076
10 1500
11 1000
12 1015


In [43]:
# not required for the script, getting merged solution dict calculating the average recall/coverage metric.
metric_dict_vote = {}
metric_dict_bm25 = {}
for (id_a, dense_list), (id_b, bm25_list) in zip(solution_dict_fin.items(), solution_dict.items()):
    # 1.
    d_list = []
    for h_ in dense_list:
        d_= h_.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)
    metric_dict_vote[id_a] = d_list
    # 2.
    d_list = []
    for h_ in bm25_list:
        d_= h_.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)
    metric_dict_bm25[id_b] = d_list

In [44]:
# not required for the script, metric calculation part.
i = 0
for x, y in metric_dict_vote.items():
    print(x, len(y))
    if i >= 10:
        break
    i = i+1
print('\n')
i = 0
for x, y in metric_dict_bm25.items():
    print(x, len(y))
    if i >= 10:
        break
    i = i+1

1 458
2 482
3 354
4 508
5 718
6 681
7 420
8 693
9 459
10 741
11 444


1 687
2 694
3 443
4 729
5 912
6 626
7 513
8 1018
9 565
10 644
11 548


In [45]:
# not required for the script, metric calculation part.
# loading the baseline qrel files.
new_rel_2021 = pd.read_csv('/content/drive/MyDrive/Touche/touche_ground_truth.csv')
new_rel_2021.head()

,qid,no,doc,rel
0,1,0,clueweb12-0001wb-05-12311,0
1,1,0,clueweb12-1811wb-62-08424,1
2,1,0,clueweb12-1811wb-62-08423,1
3,1,0,clueweb12-1217wb-47-14048,0
4,1,0,clueweb12-1811wb-62-08425,1


In [46]:
# not required for the script, metric calculation part.
from collections import defaultdict
ground_truth_dict = defaultdict(list)
rel0_truth_dict = defaultdict(list)
rel1_truth_dict = defaultdict(list)
rel2_truth_dict = defaultdict(list)

for i_, d_, x_ in zip(new_rel_2021['qid'], new_rel_2021['doc'], new_rel_2021['rel']):
    i_ = int(i_)
    d_ = str(d_)    
    if int(x_) > 0:
        ground_truth_dict[i_].append(d_)
    if int(x_) == 0:
        rel0_truth_dict[i_].append(d_)
    if int(x_) == 1:
        rel1_truth_dict[i_].append(d_)
    if int(x_) == 2:
        rel2_truth_dict[i_].append(d_)

In [48]:
# not required for the script, metric calculation part.
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 100
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_vote.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_vote.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_vote.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_vote.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.25
Average common ratio: 0.8522
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.05
Zero Relevance, Average common ratio: 0.7037
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.34
One Relevance, Average common ratio: 0.8212
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.7983


In [49]:
# not required for the script, metric calculation part.
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
# not required for the script, metric calculation part.
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 100
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.28
Average common ratio: 0.8706
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.09
Zero Relevance, Average common ratio: 0.7526
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.38
One Relevance, Average common ratio: 0.8462
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8017
